In [1]:
import pandas as pd
# pd.options.io.parquet.engine
import json


In [121]:
df = pd.read_parquet('../data/train_2wiki.parquet', engine='pyarrow')
df.head()

,_id,type,question,context,supporting_facts,evidences,answer
0,13f5ad2c088c11ebbd6fac1f6bf848b6,bridge_comparison,Are director of film Move (1970 Film) and dire...,"[[""Stuart Rosenberg"", [""Stuart Rosenberg (Augu...","[[""Move (1970 film)"", 0], [""M\u00e9diterran\u0...","[[""Move (1970 film)"", ""director"", ""Stuart Rose...",no
1,3057c6c4086111ebbd5dac1f6bf848b6,bridge_comparison,Do both films The Falcon (Film) and Valentin T...,"[[""The Falcon Takes Over"", [""The Falcon Takes ...","[[""The Falcon (film)"", 0], [""Valentin the Good...","[[""The Falcon (film)"", ""director"", ""Vatroslav ...",no
2,89bc944808a111ebbd79ac1f6bf848b6,bridge_comparison,"Which film whose director is younger, Charge I...","[[""Danger: Diabolik"", [""Danger:"", ""Diabolik i...","[[""Charge It to Me"", 1], [""Danger: Diabolik"", ...","[[""Charge It to Me"", ""director"", ""Roy William ...",Danger: Diabolik
3,633f80660bdd11eba7f7acde48001122,compositional,What is the date of birth of Mina Gerhardsen's...,"[[""Pamela Jain"", [""Pamela Jain is an Indian pl...","[[""Mina Gerhardsen"", 1], [""Rune Gerhardsen"", 0]]","[[""Mina Gerhardsen"", ""father"", ""Rune Gerhardse...",13 June 1946
4,2dc3f9740bda11eba7f7acde48001122,compositional,What nationality is the director of film Weddi...,"[[""Weekend in Paradise (1931 film)"", [""Weekend...","[[""Wedding Night in Paradise (1950 film)"", 0],...","[[""Wedding Night in Paradise"", ""director"", ""G\...",Hungarian


In [122]:
# correct encoding (M\u00e9diterran\u00e9e instead of Méditerranée)
df['supporting_facts'] = df['supporting_facts'].apply(json.loads)
df['context'] = df['context'].apply(json.loads)
df['nb_pos'] = df.supporting_facts.apply(lambda w: len(w)) # knopw how many positive docs. 

In [123]:
def ordering_context(context, support):
    first_order = []
    last_order = []
    support = [elem[0] for elem in support]
    for elem in context: 
        if elem[0] in support: 
            first_order.append(elem)
        else: 
            last_order.append(elem)
    first_order.extend(last_order)
    return first_order

In [124]:
# have positive documents in the first part. 
df.context = df[['context', 'supporting_facts']].apply(lambda x: ordering_context(x['context'], x['supporting_facts']), axis = 1) 

In [125]:
df.context = df.context.apply(lambda x: sum((i for elem in x for i in elem if isinstance(i,list)), []))

In [126]:
df.head()

,_id,type,question,context,supporting_facts,evidences,answer,nb_pos
0,13f5ad2c088c11ebbd6fac1f6bf848b6,bridge_comparison,Are director of film Move (1970 Film) and dire...,"[Stuart Rosenberg (August 11, 1927 – March 15,...","[[Move (1970 film), 0], [Méditerranée (1963 fi...","[[""Move (1970 film)"", ""director"", ""Stuart Rose...",no,4
1,3057c6c4086111ebbd5dac1f6bf848b6,bridge_comparison,Do both films The Falcon (Film) and Valentin T...,[Vatroslav Mimica( born 25 June 1923) is a Cro...,"[[The Falcon (film), 0], [Valentin the Good, 0...","[[""The Falcon (film)"", ""director"", ""Vatroslav ...",no,5
2,89bc944808a111ebbd79ac1f6bf848b6,bridge_comparison,"Which film whose director is younger, Charge I...","[Danger:, Diabolik is a 1968 action film dire...","[[Charge It to Me, 1], [Danger: Diabolik, 1], ...","[[""Charge It to Me"", ""director"", ""Roy William ...",Danger: Diabolik,4
3,633f80660bdd11eba7f7acde48001122,compositional,What is the date of birth of Mina Gerhardsen's...,[Mina Gerhardsen (born 14 September 1975) is a...,"[[Mina Gerhardsen, 1], [Rune Gerhardsen, 0]]","[[""Mina Gerhardsen"", ""father"", ""Rune Gerhardse...",13 June 1946,2
4,2dc3f9740bda11eba7f7acde48001122,compositional,What nationality is the director of film Weddi...,[Wedding Night in Paradise is a 1950 West Ger...,"[[Wedding Night in Paradise (1950 film), 0], [...","[[""Wedding Night in Paradise"", ""director"", ""G\...",Hungarian,2


In [133]:
df[['question', 'context', 'answer']].loc[:1000].to_csv('../data/2wiki_1k.csv', index = False)

In [81]:
df.supporting_facts.loc[0]

[['Move (1970 film)', 0],
 ['Méditerranée (1963 film)', 0],
 ['Stuart Rosenberg', 0],
 ['Jean-Daniel Pollet', 0]]

In [ ]:
df.drop(columns = ['id', 'type', '']) 

,_id,type,question,context,supporting_facts,evidences,answer
0,13f5ad2c088c11ebbd6fac1f6bf848b6,bridge_comparison,Are director of film Move (1970 Film) and dire...,"[Stuart Rosenberg (August 11, 1927 – March 15,...","[[""Move (1970 film)"", 0], [""M\u00e9diterran\u0...","[[""Move (1970 film)"", ""director"", ""Stuart Rose...",no
1,3057c6c4086111ebbd5dac1f6bf848b6,bridge_comparison,Do both films The Falcon (Film) and Valentin T...,[The Falcon Takes Over( also known as The Falc...,"[[""The Falcon (film)"", 0], [""Valentin the Good...","[[""The Falcon (film)"", ""director"", ""Vatroslav ...",no
2,89bc944808a111ebbd79ac1f6bf848b6,bridge_comparison,"Which film whose director is younger, Charge I...","[Danger:, Diabolik is a 1968 action film dire...","[[""Charge It to Me"", 1], [""Danger: Diabolik"", ...","[[""Charge It to Me"", ""director"", ""Roy William ...",Danger: Diabolik
3,633f80660bdd11eba7f7acde48001122,compositional,What is the date of birth of Mina Gerhardsen's...,"[Pamela Jain is an Indian playback singer., Da...","[[""Mina Gerhardsen"", 1], [""Rune Gerhardsen"", 0]]","[[""Mina Gerhardsen"", ""father"", ""Rune Gerhardse...",13 June 1946
4,2dc3f9740bda11eba7f7acde48001122,compositional,What nationality is the director of film Weddi...,[Weekend in Paradise( German: Wochenend im Par...,"[[""Wedding Night in Paradise (1950 film)"", 0],...","[[""Wedding Night in Paradise"", ""director"", ""G\...",Hungarian
...,...,...,...,...,...,...,...
167449,56100d300bdc11eba7f7acde48001122,compositional,What is the place of birth of the director of ...,[S.N. Mathur was the Director of the Indian In...,"[[""Rolling in Money"", 0], [""Albert Parker (dir...","[[""Rolling in Money"", ""director"", ""Albert Park...",New York
167450,3df1a97108ad11ebbd83ac1f6bf848b6,comparison,"Who was born first, Dušan Ninić or Eszter Balint?",[Thomas Eastwood Dickinson( 11 January 1931 – ...,"[[""Du\u0161an Nini\u0107"", 0], [""Eszter Balint...","[[""Du\u0161an Nini\u0107"", ""date of birth"", ""S...",Dušan Ninić
167451,8be4ef3e0bdc11eba7f7acde48001122,compositional,When did the director of film Morchha die?,[Thomas Scott( 1907- date of death unknown) wa...,"[[""Morchha"", 0], [""Ravikant Nagaich"", 0]]","[[""Morchha"", ""director"", ""Ravikant Nagaich""], ...",6 January 1991
167452,12357df20bdc11eba7f7acde48001122,compositional,What is the date of birth of the director of f...,"[Peter Levin is an American director of film, ...","[[""Double Cross (1951 film)"", 0], [""Riccardo F...","[[""Double Cross"", ""director"", ""Riccardo Freda""...",24 February 1909


In [2]:
pd.read_csv('../Experiment_data/2wiki/results_SENTENCE.csv')

,query,context,provided_answer,lds,lds_compar,rmse,r2,topk_probability,topk_divergence,method_scores
0,Do both Something Always Happens and The Great...,[[['Michael Latham Powell( 30 September 1905 –...,"No, they do not. \n\nSomething Always Happens ...","[[[{'ContextCite364': 0.8055617352614016}, {'F...","[[[{'FM_WeightsD364': 0.8771968854282535}, {'F...","[[[{'ContextCite364': 3.0445518775712443}, {'F...","[[[{'ContextCite364': 0.6177416901646957}, {'F...","[{'ContextCite364': {1: 13.729990005493164, 2:...","[{'ContextCite364': {1: 1.9163229748886579, 2:...","[{'ContextCite364': array([-0.15105068, -0. ..."
